In [1]:
import re
import pandas as pd 
import numpy as np

In [2]:
Questions2012 = pd.read_csv('../../160-Stackoverflow-Data/train_test/X_train.csv' ,\
                    dtype = {'LastEditorDisplayName': str, 'Tags': str})

In [ ]:
Questions2012

,Unnamed: 0,Unnamed: 0.1,Id,ParentId,AcceptedAnswerId,CreationDate,Score,ViewCount,Body,OwnerUserId,OwnerDisplayName,LastEditorUserId,Title,Tags,AnswerCount,CommentCount,FavoriteCount,ClosedDate
0,189467,2127191,12317590,NaN,NaN,2012-09-07T11:53:12.047,0,437.0,<p>I have successfully implemented and deploye...,1444886,NaN,NaN,Java JDK versions eclipse Blackboard 9.1 java....,eclipse java blackboard,1.0,4,NaN,NaN
1,245334,2736003,13382267,NaN,13401477.0,2012-11-14T15:52:20.880,1,532.0,<p>I'm developing an application integrating S...,1824054,NaN,NaN,@Cacheable from Spring 3.1 implementation inco...,java spring annotations,1.0,4,NaN,NaN
2,183601,2062961,12206490,NaN,12887712.0,2012-08-30T22:29:47.340,6,330.0,"<p>I came across this <a href=""https://gist.gi...",473709,NaN,473709.0,Is document.getElementById('id') or $('#id') s...,javascript html dom,4.0,2,3.0,NaN
3,134931,1528087,11283530,NaN,NaN,2012-07-01T16:05:31.943,0,991.0,<p>Currently I am writing to a file on my loca...,982026,NaN,982026.0,Write file to another computer in local networ...,java javascript file firefox networking,0.0,14,0.0,NaN
4,141856,1602774,11412528,NaN,NaN,2012-07-10T11:38:34.003,3,456.0,"<p>I have about <code>15,000</code> records of...",1472699,NaN,750613.0,Android SQLite query to get closest location t...,java android sqlite,0.0,3,NaN,NaN
5,218886,2451911,12885638,NaN,12885815.0,2012-10-14T19:18:19.410,-1,199.0,"<p>I have problem with script, can someone hel...",1378518,NaN,1378518.0,Google Maps v3 event click in cycle,javascript jquery events,1.0,4,0.0,NaN
6,174833,1965853,12038962,NaN,12039011.0,2012-08-20T13:51:02.190,2,420.0,<p>I have following code:</p>\r\r\r\n\r\r\r\n<...,610641,NaN,610641.0,"Why XMLHttpRequest doesn't work with ""www.""?",javascript jquery ajax redirect,2.0,3,NaN,NaN
7,27053,323956,9232733,NaN,9232788.0,2012-02-10T18:19:33.427,2,2467.0,<p>I work on a relatively large project that h...,38971,NaN,NaN,Seeing Code Coverage for a Single Class,java emma,2.0,5,NaN,NaN
8,109482,1244963,10796495,NaN,10796557.0,2012-05-29T09:34:16.363,3,1371.0,<p>I need check with junit that method of nest...,810430,NaN,1836.0,How to test anonymous nested class in java?,java class testing junit nested,4.0,0,NaN,NaN
9,202156,2266166,12562172,NaN,NaN,2012-09-24T09:24:41.200,1,904.0,"<p>For my current project (<a href=""http://bet...",1374678,NaN,1374678.0,Standard Config File Location for a Java Library?,java configuration resourcebundle,2.0,1,NaN,NaN


In [3]:
Answers2012 = pd.read_csv('../../160-Stackoverflow-Data/train_test/Answers.csv' ,\
                    dtype = {'LastEditorDisplayName': str, 'Tags': str})

In [4]:
Answers2012.tail()

,Id,ParentId,CreationDate,Score,ViewCount,Body,OwnerUserId,ClosedDate
2824385,14106907,14106883.0,2012-12-31T23:58:26.823,2,NaN,<p>You only take 1 item from user so no need t...,1906557,NaN
2824386,14159856,14159855.0,2012-12-20T11:23:10.723,3,NaN,<p>One way is to use the <code>ErrorT</code> m...,784338,NaN
2824387,14159857,14159855.0,2012-12-20T14:04:42.787,0,NaN,<p>This is what I have came up with </p>\r\n\r...,439578,NaN
2824388,14159858,14159855.0,2012-12-20T17:58:07.890,6,NaN,<p>A slightly more sensible approach in Haskel...,157360,NaN
2824389,14525989,14525988.0,2012-12-01T17:50:01.187,1,NaN,<p>When you click the <kbd>F12</kbd> in Dreamw...,724764,NaN


In [ ]:
Questions2012[Questions2012['Id'] == 12206490]

In [ ]:
Answers2012[Answers2012['ParentId'] == 12206490]

## Frequency table of tags

In [5]:
# make a function that extracts tags from Questions' tags, using Questions' Id
def freq_table(idx):
    '''
    @param idx: list
    return series
    '''
    filtered = [] 
    for j in range(len(idx)):
        tags = Questions2012.loc[(Questions2012.Id == idx.iloc[j])]['Tags'] #extracting tags from Questions' Id
        
        try:
            filtered.extend(Questions2012.loc[(Questions2012.Id == idx.iloc[j])]['Tags'].iloc[0].split(" "))
        except:
            continue
    freq_tags = pd.DataFrame({'Tag':filtered}).groupby('Tag').size()
    
    return freq_tags

### Tags of a user's answers

In [6]:
#tags of 'question' posts that the user posted 'answers' to, and count the number of the user's answers for each tag
def answers_user(user_id):
    """
    @param user_id: int 
    return: series 
    """
    idx = Answers2012.loc[(Answers2012.OwnerUserId == user_id)]['ParentId']
    return freq_table(idx)

In [7]:
#example
answers_user(476)

Tag
ajax                        2
alert                       2
android                     1
arrays                      3
authentication              1
caching                     1
cakephp                     1
closures                    2
cookies                     1
debugging                   1
file                        2
function                    1
header                      2
html                        2
html5                       1
http                        1
ide                         1
iframe                      1
ip-address                  1
java                        4
javascript                 63
jquery                      9
json                        8
jsonp                       1
jsp                         2
loops                       1
mime                        1
mysql                       3
object                      1
pdo                         1
php                        33
python                      1
regex                       2
scope 

In [ ]:
answers_user(476).index

### Acceptance ratio of a user's answers

In [ ]:
# do not include in activities function yet

In [8]:
#tags of 'question' posts that the user's answer was accepted to('AcceptedAnswer'), and count the number of the user's accepted answers for each tag
def accepted_ratio(user_id):
    """
    @param user_id: int 
    return: series 
    """
    accepted_tags = []
    answers = Answers2012.loc[(Answers2012.OwnerUserId == user_id) ]

    for i in range(len(answers)):
        tags = Questions2012.loc[(Questions2012.AcceptedAnswerId == answers.Id.iloc[i])]['Tags']
        try:
            accepted_tags.extend(tags.iloc[0].split(" "))
        except:
            continue
    freq_acanswers = pd.DataFrame({'Tag':accepted_tags}).groupby('Tag').size()
    return round(freq_acanswers/answers_user(user_id),2).dropna()

In [9]:
#example
accepted_ratio(724764)

Tag
checkbox           1.00
contenteditable    1.00
cookies            0.50
cursor             1.00
dom                0.50
fadein             1.00
html               0.13
javascript         0.11
jquery             0.13
kml                1.00
setcookie          1.00
show-hide          1.00
dtype: float64

### Tags of a user's questions

In [10]:
def question_user(user_id):
    """
    @param user_id: int 
    return: series 
    """
    idx = Questions2012.loc[(Questions2012.OwnerUserId == user_id)]['Id']
    return freq_table(idx)

In [11]:
question_user(982026)

Tag
css           1
events        1
file          1
firefox       1
forms         1
html          1
java          2
javascript    3
networking    1
triggers      1
dtype: int64

### Tags of a user's comments

In [12]:
Comments = pd.read_csv('../../160-Stackoverflow-Data/train_test/Comments.csv')

In [13]:
Comments.head()

,Unnamed: 0,Id,PostId,Score,Text,CreationDate,UserId,UserDisplayName
0,0,10809358,8691122,1,@jcubic injection FTW. Sanitize your variables :P,2012-01-01T00:00:12.463,445887.0,NaN
1,1,10809359,8691108,0,One of the criteria for the menu item to be vi...,2012-01-01T00:00:28.220,418957.0,NaN
2,2,10809360,8690172,0,@Ben What predictive algorithm does a modern i...,2012-01-01T00:00:33.020,53543.0,NaN
3,3,10809362,8690991,0,What _is_ the `redirect_uri` you're specifying...,2012-01-01T00:00:44.897,377270.0,NaN
4,4,10809363,8691095,0,Please add a link to the manual you talk about...,2012-01-01T00:00:55.457,367456.0,NaN


In [14]:
#tags of 'question' posts that the user posted 'comments' to, and count the number of the user's comments for each tag
def comments_user(user_id):
    """
    @param user_id: int 
    return: series 
    """
    idx = Comments.loc[(Comments.UserId == user_id)]['PostId']
    return freq_table(idx)

In [15]:
#example
comments_user(53543)

Tag
java    1
xna     1
dtype: int64

## Total number of activities against tags

In [16]:
# activities include answers, accepted answers and comments
def activities(user_id):
    """
    @param user_id: int 
    return: dataframe
    """
    answers = answers_user(user_id)
    acc_ratio = accepted_ratio(user_id)
    comments = comments_user(user_id)
    questions = question_user(user_id)
    user_table = pd.DataFrame({'Answers' : answers ,'Acceptance Ratio' : acc_ratio ,'Questions':questions,'Comments': comments})
    user_table = user_table.fillna(0)
    if len(user_table) != 0 :
        user_table['Expertise'] = user_table.apply(lambda 
                                                  row: (row['Answers']+1/2*row['Comments']+1/2*row['Questions']
                                                       )*(row['Acceptance Ratio']), axis=1) #*(row['Acceptance Ratio'])
        # As comments are less important than answers or accepted answers, adjusting the impact of comments
        user_table = user_table[user_table.Expertise > user_table.Expertise.quantile(0.25)] # drop lower 25% strength values 
        user_table.Expertise = user_table.Expertise / sum(user_table.Expertise) #scaling
    return user_table

In [17]:
activities(377270)

,Acceptance Ratio,Answers,Comments,Questions,Expertise
Tag,,,,,
aes,1.00,1.0,2.0,0.0,0.035628
antlr,1.00,1.0,1.0,0.0,0.026721
core,1.00,1.0,0.0,0.0,0.017814
expression,1.00,1.0,0.0,0.0,0.017814
ftp,1.00,1.0,2.0,0.0,0.035628
grammar,1.00,1.0,0.0,0.0,0.017814
java,0.36,28.0,98.0,0.0,0.493810
javascript,0.17,6.0,37.0,0.0,0.074196
jvm,1.00,1.0,0.0,0.0,0.017814


In [ ]:
activities(367456)

In [ ]:
activities(53543)